In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, gc, random
import keras
pd.set_option('max_columns', None)
from scipy import stats
import scipy
import tensorflow as tf
plt.style.use('fivethirtyeight')
from tqdm.notebook import tqdm
from sklearn import preprocessing, model_selection
from colorama import Fore, Back, Style
from IPython.display import clear_output
%matplotlib inline

In [ ]:
try:
    import holidays
except:
    !pip install holidays
    
clear_output()

<h1 style="color:#FF4533;font-size:50px;"><strong>TPS <strong style="color:#337CFF"> JAN 2022</strong></strong></h1>


![](https://res.cloudinary.com/teepublic/image/private/s--LYq6T5Y0--/t_Resized%20Artwork/c_crop,x_10,y_10/c_fit,w_470/c_crop,g_north_west,h_626,w_470,x_0,y_0/g_north_west,u_upload:v1462829017:production:blanks:qe3008lhp5hquxmwp4a0,x_-395,y_-325/b_rgb:eeeeee/c_limit,f_auto,h_630,q_90,w_630/v1521830347/production/designs/2522554_0.jpg)

In [ ]:
#funtion to clean plot

#github: https://gist.github.com/Ankitkalauni/093ddbd4435246ec1826598db4a67e9d

def setup_plot(w=1,h=1,rows=1,cols=1,X_label='\n',y_label='\n',dpi=600, color = None):
    '''
    w: width of the figsize parameter of matplotlib.pyplot
    h: height of the figsize parameter of matplotlib.pyplot
    rows: numbers of rows in gridspec
    cols: numbers of cols in gridspec
    X_label: Label of the x-axis
    y_label: Label of the y-axis
    dpi: to set dpi for the figure.dpi
    color: if None the colors of bar will different. if not None(pass any value) all bars colors will be same.
    
    Return axes (only single axes for now, still working on the multiple axes)
    
    design by @ankitkalauni
    '''
    #setting plot theme
    plt.rcParams['figure.dpi'] = dpi

    fig = plt.figure(figsize=(w,h), facecolor='#f6f5f5')
    gs = fig.add_gridspec(rows, cols)
    gs.update(wspace=1.5, hspace=0.05)
    
    PAL = ['#336b87', '#b4b4b4','#dd4124','#009473',
           '#F1C40F','#DFFF00','#40E0D0','#6495ED',
           '#FF0000','#00FF00','#0000FF','#FF00FF','#800080']
    
    if color:
        random = np.random.randint(low = 0,high = len(PAL))
        PAL = [PAL[random] for _ in range(0,120)]

    background_color = "#f6f5f5"
    sns.set_palette(PAL)
    
    #making single ax
    ax_dict = {}
    for row in range(rows):
        for col in range(cols):
            ax_dict["ax%s%s" %(row,col)] = fig.add_subplot(gs[row, col])

    locals().update(ax_dict)

    #setting theme for every ax in local()
    for row in range(rows):
        for col in range(cols):
                    
            for s in ["right", "top"]:
                locals()['ax' + str(row) + str(col)].spines[s].set_visible(False)
                
            locals()['ax' + str(row) + str(col)].set_facecolor(background_color)

            locals()['ax' + str(row) + str(col)].set_facecolor(background_color)
            locals()['ax' + str(row) + str(col)].set_xlabel(X_label,fontsize=3, weight='bold')
            locals()['ax' + str(row) + str(col)].set_ylabel(y_label,fontsize=3, weight='bold')

            locals()['ax' + str(row) + str(col)].tick_params(labelsize=3, width=0.5, length=1.5)
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='x', zorder=0, color='#EEEEEE', linewidth=0.4)
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='y', zorder=0, color='#EEEEEE', linewidth=0.4)

    return locals()['ax' + str(row) + str(col)]


<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
1. Version Control
</div>



<br>

<mark>v1</mark> -> FAILED

<mark>v2</mark> -> Getting Started with my first time series dataset - Data Viz

<mark>v3</mark> -> Swish Robustscaler 2 layer DNN, no FE - oof DNN: 64.15303039550781 public: 16.7

<mark>v4</mark> -> RELU Standardscaler 3 layer DNN, no FE - oof DNN: 65.89269256591797 public: 15.9

<mark>v5</mark> -> Fixed Columns and make spare matrix - 10. public lb 

<mark>v6</mark> -> Cyclic time measure - 11. public lb

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
2. Findings
    
</div>


<br>



<mark>Period</mark>: 2015 - 2018

<mark>Stores</mark>: KaggleRama, KaggleMart

<mark>Countries</mark>: Finland, Sweden, Norway 

<mark>Products</mark>: Kaggle Mug, Kaggle Hat, Kaggle Sticker

___

Most sold is Kaggle Sticker by KaggleRama in every country given. i am lazy

In [ ]:
working_dir = '../input/tabular-playground-series-jan-2022/'

train = pd.read_csv(f'{working_dir}train.csv')
test = pd.read_csv(f'{working_dir}test.csv')

sample_submission = pd.read_csv(f'{working_dir}sample_submission.csv')

In [ ]:
#knobs
to_plot = False
to_train = True
cyclic = False

In [ ]:
def view_data(df):
    print("DataFrame has shape of: ",df.shape)
    print('='*60)
    print('Dataframe Info')
    display(df.info())
    print('='*60)
    print('Dataframe Describe')
    display(df.describe().T)
    print('='*60)
    print('Number of Null Values:', df.isnull().sum().sum())
    print('='*60)
    print(f'''
    number of columns having Int dtypes: {train.select_dtypes('int').shape[-1]}\n
    number of columns having Float dtypes: {train.select_dtypes('float').shape[-1]}\n
    number of columns having Object dtypes: {train.select_dtypes('object').shape[-1]}\n
    ''')
    
    print('='*60)
    display(df.head())

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
3.EDA
</div>
<br>

In [ ]:
if to_plot:
    display(view_data(train))
    print(Back.GREEN)
    print('-'*130)
    print('= END ='*20)
    print('-'*130)
    print(Style.RESET_ALL)
    view_data(test)

In [ ]:
to_drop = ['row_id']

def get_date(df):
    df[['year','month', 'day']] = df['date'].str.split('-', expand=True).astype('int32').values #spliting column date
    
    df.drop(to_drop, axis=1, inplace=True) #droping useless columns
    
    return df

train = get_date(train)
test = get_date(test)

In [ ]:
if to_plot:
    sns.kdeplot(data = train,x='num_sold', hue='country', ax = setup_plot(w=6, h=3, X_label = 'num_sold'), fill=True, alpha=0.6, zorder=5)
    display(train.groupby(by='country')['num_sold'].sum())
    plt.title('No. of Sold by Countries', fontsize=8)
    sns.barplot(data = train,y='num_sold',x='country', ax = setup_plot(w=6, h=3, X_label = 'country'), fill=True, alpha=0.6, zorder=5)

In [ ]:
if to_plot:
    sns.kdeplot(data = train,x='num_sold', hue='product', ax = setup_plot(w=6, h=3, X_label = 'num_sold'), fill=True, alpha=0.6, zorder=5);
    plt.title('No. of Sold by Products', fontsize=8)
    sns.barplot(data = train,y='num_sold',x='country', hue='product', ax = setup_plot(w=6, h=3, X_label = 'country'), fill=True, alpha=0.6, zorder=5)
    plt.legend(loc='best', fontsize='small');

In [ ]:
if to_plot:
    sns.kdeplot(data = train,x='num_sold', hue='store', ax = setup_plot(w=6, h=3, X_label = 'num_sold'), fill=True, alpha=0.6, zorder=5)
    plt.title('No. of Sold by Stores', fontsize=8)
    sns.barplot(data = train,y='num_sold',x='country', hue='store', ax = setup_plot(w=6, h=3, X_label = 'country'), fill=True, alpha=0.6, zorder=5)
    plt.legend(loc='best', fontsize='small');

In [ ]:
if to_plot:    
    sns.barplot(data = train,y='num_sold',x='product', hue='store', ax = setup_plot(w=6, h=3, X_label = 'product'), fill=True, alpha=0.6, zorder=5)
    plt.legend(loc='best', fontsize='small');
    plt.title('Product-wise No. of Sold by Stores', fontsize=8)

In [ ]:
if to_plot:
    sns.lineplot(data = train,x = 'date', y = 'num_sold', hue = 'country', ax = setup_plot(w=8, h=3,X_label='Date', y_label = 'num_sold'), zorder = 5)
    plt.legend(loc='best', fontsize='small'); #long time to run
    plt.tick_params(bottom = False,labelbottom = False)

In [ ]:
_train = train.loc[train['product'] == 'Kaggle Sticker'].copy()

if to_plot:
    sns.histplot(data = _train,x='num_sold', hue='store', ax = setup_plot(w=6, h=3, X_label = 'num_sold'), alpha=0.6, zorder=5)
    display(_train.groupby(by = 'store')['num_sold'].sum())
    plt.title('No. of Sold of Kaggle Sticker by Stores', fontsize=8)
    
del _train
gc.collect()

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
4. Preprocessing
    
</div>

In [ ]:
if not to_train:
    bruh

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DateProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, date_format='%Y-%m-%d', hours_secs=False):
        self.format = date_format
        self.columns = None
        self.time_transformations = [
            ('day_sin', lambda x: np.sin(2*np.pi*x.dt.day/31)),
            ('day_cos', lambda x: np.cos(2*np.pi*x.dt.day/31)),
            ('dayofweek_sin', 
                lambda x: np.sin(2*np.pi*x.dt.dayofweek/6)),
            ('dayofweek_cos', 
                lambda x: np.cos(2*np.pi*x.dt.dayofweek/6)),
            ('month_sin', 
                lambda x: np.sin(2*np.pi*x.dt.month/12)),
            ('month_cos', 
                lambda x: np.cos(2*np.pi*x.dt.month/12)),
            ('year', 
                lambda x: (x.dt.year - x.dt.year.min()
                          ) / (x.dt.year.max() - x.dt.year.min()))
        ]
        if hours_secs:
            self.time_transformations = [
                ('hour_sin', 
                lambda x: np.sin(2*np.pi*x.dt.hour/23)),
                ('hour_cos', 
                lambda x: np.cos(2*np.pi*x.dt.hour/23)),
                ('minute_sin', 
                lambda x: np.sin(2*np.pi*x.dt.minute/59)),
                ('minute_cos', 
                lambda x: np.cos(2*np.pi*x.dt.minute/59))
            ] + self.time_transformations

    def fit(self, X, y=None, **fit_params):
        self.columns = self.transform(X.iloc[0:1,:]).columns
        return self

    def transform(self, X, y=None, **fit_params):
        transformed = list()
        for col in X.columns:
            time_column = pd.to_datetime(X[col],
                              format=self.format)
            for label, func in self.time_transformations:
                transformed.append(func(time_column))
                transformed[-1].name += '_' + label
        transformed = pd.concat(transformed, axis=1)
        return transformed

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

In [ ]:
if cyclic:
    test['num_sold'] = -1

    all_data = pd.concat([train, test], axis=0)

    temp = DateProcessor().fit_transform(all_data[['date']])

    all_data = pd.concat([all_data, temp], axis=1)

    train = all_data.loc[all_data['num_sold'] != -1, :]
    test = all_data.loc[all_data['num_sold'] == -1, :]
    
    test = test.drop(columns = ['num_sold'])

In [ ]:
train.shape, test.shape

In [ ]:
#thanks to cv13j0 for teaching me this preprocessing for time series data

def calender(df):
    '''preprocess the date column and getting more information from
    that column'''
    
    df['date'] = pd.to_datetime(df['date']) #converting to datetime dtype
#     df.index = df['date']
    df['quater'] = df['date'].dt.quarter
    df['day_of_week'] = df['date'].dt.day_of_week
    df['day_of_year'] = df['date'].dt.day_of_year
    df['days_in_month'] = df['date'].dt.days_in_month
    df['is_leap_year'] = df['date'].dt.is_leap_year.astype('int32')
    df['week'] = df['date'].dt.week
    df['is_weekend'] = 0 
    df.loc[(df['day_of_week'] == 6) | (df['day_of_week'] == 5), ['is_weekend']] = 1
    
    return df

In [ ]:
train = calender(train)
test = calender(test)

clear_output()

In [ ]:
if to_plot:
    display(train)
    display(test)

In [ ]:
def chutii(df):
    df = df.reset_index(drop=True)
    countries = df['country'].unique().tolist()
    df['is_holiday'] = 0
    for count in range(0, len(countries)):
        print('Imputing Holidays for {} Country'.format(countries[count]))
        temp = df.loc[train['country'] == countries[count]]
        chutii = holidays.CountryHoliday(countries[count])
        temp['is_holiday'] = temp['date'].apply(lambda x: np.where(x in chutii, 1, 0))  
        df.loc[df['country'] == countries[count], 'is_holiday'] = temp['is_holiday']
            
    return df

In [ ]:
train = chutii(train)
test = chutii(test)

clear_output()

In [ ]:
temp1 = np.log(train['num_sold'])
temp2 = stats.boxcox(train['num_sold'])[0]

bro = sns.kdeplot(temp1, color = 'red')
sns.kdeplot(temp2, color = 'blue', ax = bro)

plt.show();

In [ ]:
from scipy import stats
y, lambda_ = stats.boxcox(train['num_sold'].astype('float32'))
y = pd.Series(y)
# train.index = train['date']
# test.index = test['date']

X = train.drop(columns = ['num_sold'])


In [ ]:
cat_cols = ['country', 'store', 'product', 'year', 'month', 'day', 'quater', 'day_of_week', 'day_of_year', 'days_in_month', 'week']
to_keep = ['is_leap_year','is_weekend','is_holiday']

In [ ]:
X.shape, test.shape

In [ ]:
all_data = pd.concat([X, test],axis=0).reset_index(drop=True)

for col in tqdm(cat_cols):
    temp = pd.get_dummies(all_data[col], prefix='_'+col, drop_first=True)
    all_data = pd.concat([all_data, temp], axis=1)
    
all_data = all_data.drop(columns = cat_cols)

In [ ]:
X = all_data.loc[0:X.shape[0]-1,:]
test = all_data.loc[X.shape[0]:,:]

In [ ]:
X.index = X['date']
test.index = test['date']

X = X.drop(columns = ['date'])
test = test.drop(columns = ['date'])

In [ ]:
X.shape, test.shape

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
5. Model Training
    
</div>

In [ ]:
from keras.layers import Dense, BatchNormalization, Input, Concatenate, Multiply, Activation, Dropout, Add
from keras import Model

def get_model(SEED):
    tf.keras.backend.clear_session()
    np.random.seed(SEED)
    random.seed(SEED)
    tf.random.set_seed(SEED)
    
    input_ = Input(shape = (X.shape[-1],))
        
    x = Dense(1024, kernel_initializer='he_normal')(input_)
    x = Activation('swish')(x)
    x = Dropout(0.45)(x)
    
#     x1 = Dense(150, kernel_initializer='he_normal')(x)
#     x1 = BatchNormalization()(x1)
#     x1 = Activation('swish')(x1)
#     x1 = Dropout(0.55)(x1)
    
#     x2 = Dense(71, kernel_initializer='he_normal')(x1)
#     x2 = BatchNormalization()(x2)
#     x2 = Activation('swish')(x2)
#     x2 = Dropout(0.45)(x2)
    
#     x3 = Dense(150, kernel_initializer='he_normal')(x2)
#     x3 = BatchNormalization()(x3)
#     x3 = Activation('swish')(x3)
#     x3 = Multiply()([x1, x3])
#     x3 = Dropout(0.55)(x3)

    
#     x3 = Dense(71, kernel_initializer='he_normal')(x3)
#     x3 = BatchNormalization()(x3)
#     x3 = Multiply()([x2, x3])
#     x3 = Activation('swish')(x3)
#     x3 = Dropout(0.45)(x3)
    
    x1 = Dense(228, kernel_initializer='he_normal')(input_)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Dropout(0.15)(x1)
    
    x2 = Dense(124, kernel_initializer='he_normal')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = Dropout(0.15)(x2)
    
    x3 = Dense(64, kernel_initializer='he_normal')(x2)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)
#     x3 = Dropout(0.15)(x3)
    
    
#     x3 = Dense(21, kernel_initializer='he_normal')(x2)
# #     x3 = Concatenate()([x3,x2])
#     x3 = BatchNormalization()(x3)
#     x3 = Activation('relu')(x3)
#     x3 = Dropout(0.25)(x3)
    
    output_ = Dense(1)(x3)
    
    model = Model(inputs=[input_], outputs = [output_])
    
    return model

In [ ]:
# tf.keras.utils.plot_model(get_model(42), show_dtype=True, show_layer_names=True)

In [ ]:
class SMAPE(keras.losses.Loss):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)        

    def call(self, y_true, y_pred):
        num = tf.math.abs(tf.math.subtract(y_true, y_pred))
        denom = tf.math.add(tf.math.abs(y_true), tf.math.abs(y_pred))
        denom = tf.math.divide(denom,200.0)

        val = tf.math.divide(num,denom)
        val = tf.where(denom == 0.0, 0.0, val)
        
        return tf.reduce_mean(val)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

In [ ]:
def smape(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

# OPTUNA

___


In [ ]:
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit

# create trial function
# OPTUNA_OPTIMIZATION = True

def objective(trial, X=X, y=y):
    cv = TimeSeriesSplit(n_splits=4)

    cv_scores = []
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = preprocessing.StandardScaler()

        X_train = pd.DataFrame(columns=X_train.columns, data=scaler.fit_transform(X_train))
        X_test = pd.DataFrame(columns=X_test.columns, data=scaler.transform(X_test))
        
        params = {
            'iterations':trial.suggest_int("iterations", 10000, 20000),
            'objective': trial.suggest_categorical('objective', ['MAE']),
            'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.001,0.01),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1,15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            'verbose': False,
#             'task_type' : 'GPU',
#             'devices' : '0'
        }

        params['grow_policy'] = 'Depthwise'
        params['iterations'] = 10000

        if params['bootstrap_type'] == 'Bayesian':
            params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
        elif params['bootstrap_type'] == 'Bernoulli':
            params['subsample'] = trial.suggest_float('subsample', 0.1, 1)

        model = CatBoostRegressor(**params, eval_metric = 'MAE')
        model.fit(
            X_train, y_train,
            eval_set=[(X_test,y_test)],
            early_stopping_rounds=500,
            use_best_model=True
        )

        # validation prediction
        y_hat = model.predict(X_test)
        rmse = smape(y_test, y_hat)
        cv_scores.append(rmse)
    
    del X_train, y_train, X_test, y_test, y_hat
    gc.collect()
    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name="Catboost Classifier")
study.optimize(objective, n_trials=100,timeout=7*60*60)

In [ ]:
study.best_trial.params

In [ ]:
break

In [ ]:
# %%time

# #X, y, X_test
# from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, TimeSeriesSplit
# from sklearn.metrics import roc_curve, auc,roc_auc_score
# import lightgbm as lgb
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from lightgbm import LGBMClassifier
# from imblearn.over_sampling import SVMSMOTE as smote
# import keras
# from sklearn.ensemble import RandomForestRegressor
# from tqdm import tqdm
# from sklearn import preprocessing
# from sklearn import linear_model
# # create dictionaries to store predictions
# oof_pred_tmp = dict()
# test_pred_tmp = dict()
# scores_tmp = dict()

# name = 'dnn'

# oof_pred_tmp[name] = list()
# oof_pred_tmp['y_valid'] = list()
# test_pred_tmp[name] = list()
# scores_tmp[name] = list()


# # create cv
# kf = TimeSeriesSplit(n_splits=4)

# for fold, (idx_train, idx_valid) in enumerate(kf.split(X)):
# #     model = get_model(42)
    
# #     callback = tf.keras.callbacks.EarlyStopping(
# #         monitor='val_loss', patience=20, verbose=0,
# #         mode='min',restore_best_weights=True)
# #     reduce_lr = keras.callbacks.ReduceLROnPlateau(
# #         monitor='val_loss', 
# #         factor=0.2,
# #         patience=5,
# #         mode='min'
# #         )
    
# #     model.compile(
# #         optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
# #         loss=smape,
# #     )
    
#     # create train, validation sets
#     X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
#     X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
#     ### standardize data
#     print('scaling')
#     scaler = preprocessing.StandardScaler()

#     X_train = pd.DataFrame(columns=X_train.columns, data=scaler.fit_transform(X_train))
#     test_scaled = pd.DataFrame(columns=test.columns, data=scaler.transform(test))
#     X_valid = pd.DataFrame(columns=X_valid.columns, data=scaler.transform(X_valid))
        
# #     print('fitting')
# #     history = model.fit(X_train,y_train,validation_data=(X_valid, y_valid),
# #         callbacks=[callback,reduce_lr], epochs=1000,batch_size=2024,
# #         shuffle=True, verbose=1,validation_batch_size=len(y_valid)
# #     )
# #     print('done fitting')

# #     lol = pd.DataFrame(history.history['loss']).plot(label='loss')
# #     pd.DataFrame(history.history['val_loss']).plot(ax=lol, label='val_loss')
    
# #     plt.legend(['loss','val_loss'])

# #     plt.show()
#     # validation prediction
#     print('fitting')
#     model = lgb.LGBMRegressor(metrics='mae',verbose=0)
    
#     model.fit(X_train,y_train, eval_metric='mae', eval_set=(X_valid, y_valid), early_stopping_rounds=500)
    
#     pred_valid = model.predict(X_valid)    
    
#     score = smape(y_valid, pred_valid)

#     scores_tmp[name].append(score)
#     oof_pred_tmp[name].extend(pred_valid)

#     print(f"Fold: {fold + 1} Model: {name} Score: {score}")
#     print('--'*20)
#     # test prediction
#     y_hat = model.predict(test_scaled)

#     test_pred_tmp[name].append(y_hat)
    
    
#     print('Prediction')
    
#     # store y_validation for later use
#     oof_pred_tmp['y_valid'].extend(pd.DataFrame(y_valid,columns=['y_valid']).values.reshape(-1,1))
    
#     gc.collect()
#     # print overall validation scores
# # for name, model in models:
# print(f"Overall Validation Score | {name}: {np.mean(scores_tmp[name])}")
# print('::'*20)

In [ ]:
%%time

#X, y, X_test
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import roc_curve, auc,roc_auc_score
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SVMSMOTE as smote
import keras
from tqdm import tqdm
from sklearn import preprocessing
# create dictionaries to store predictions
oof_pred_tmp = dict()
test_pred_tmp = dict()
scores_tmp = dict()

name = 'dnn'

oof_pred_tmp[name] = list()
oof_pred_tmp['y_valid'] = list()
test_pred_tmp[name] = list()
scores_tmp[name] = list()


# create cv
kf = TimeSeriesSplit(n_splits=4)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X)):
    model = get_model(42)
    
    callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, verbose=0,
        mode='min',restore_best_weights=True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2,
        patience=5,
        mode='min'
        )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss=SMAPE(),
    )
    
    # create train, validation sets
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    ### standardize data
    print('scaling')
    scaler = preprocessing.StandardScaler()

    X_train = pd.DataFrame(columns=X_train.columns, data=scaler.fit_transform(X_train))
    test_scaled = pd.DataFrame(columns=test.columns, data=scaler.transform(test))
    X_valid = pd.DataFrame(columns=X_valid.columns, data=scaler.transform(X_valid))
        
    print('fitting')
    history = model.fit(X_train,y_train,validation_data=(X_valid, y_valid),
        callbacks=[callback,reduce_lr], epochs=1000,batch_size=2024,
        shuffle=True, verbose=1,validation_batch_size=len(y_valid)
    )
    print('done fitting')

    lol = pd.DataFrame(history.history['loss']).plot(label='loss')
    pd.DataFrame(history.history['val_loss']).plot(ax=lol, label='val_loss')
    
    plt.legend(['loss','val_loss'])

    plt.show()
    # validation prediction
    pred_valid = model.predict(X_valid)    
    
    score = SMAPE()(y_valid, pred_valid)

    scores_tmp[name].append(score)
    oof_pred_tmp[name].extend(pred_valid)

    print(f"Fold: {fold + 1} Model: {name} Score: {score}")
    print('--'*20)
    # test prediction
    y_hat = model.predict(test_scaled)

    test_pred_tmp[name].append(y_hat)
    
    
    print('Prediction')
    
    # store y_validation for later use
    oof_pred_tmp['y_valid'].extend(pd.DataFrame(y_valid,columns=['y_valid']).values.reshape(-1,1))
    
    gc.collect()
    # print overall validation scores
# for name, model in models:
print(f"Overall Validation Score | {name}: {np.mean(scores_tmp[name])}")
print('::'*20)

In [ ]:
# create df with base predictions on test_data
base_test_predictions = pd.DataFrame(
    {name: np.mean(np.column_stack(test_pred_tmp[name]), axis=1) 
    for name in test_pred_tmp.keys()}
)

# save csv checkpoint
base_test_predictions.to_csv('./base_test_predictions.csv', index=False)

# create simple average blend 
base_test_predictions['simple_avg'] = base_test_predictions.mean(axis=1)

# create submission file with simple blend average
simple_blend_submission = sample_submission.copy()
simple_blend_submission['num_sold'] = np.exp(base_test_predictions['simple_avg'])
simple_blend_submission.to_csv('simple_blend_submission.csv', index=False)


for indx, name in enumerate(oof_pred_tmp.keys()):
    temp = pd.Series(oof_pred_tmp[name], name=name)

    if indx == 0:
        df = temp
    else:
        df = pd.concat([df,temp], axis=1)

df.to_csv('oof_predictionseries.csv', index=False)

In [ ]:
import scipy
sample = sample_submission.copy()
sample['num_sold'] = np.floor(scipy.special.inv_boxcox(base_test_predictions['simple_avg'].values, lambda_))

sample.to_csv('simple_sub_floor.csv',index=False)

In [ ]:
temp = sample['num_sold']
temp.index = test.index
temp2 = scipy.special.inv_boxcox(y, lambda_)
temp2.index = X.index

temp = pd.concat([temp2, temp], axis=0)

In [ ]:
# temp.loc[temp.index.year < 2019]

# fig = plt.figure(figsize=(18,6))
temp.loc[temp.index.year < 2019].plot(figsize=(10,3),linewidth=1, color='red')
temp.loc[temp.index.year >= 2019].plot(figsize=(10,3),linewidth=1, color='green')